In [165]:
# Useful starting lines
%matplotlib inline
import numpy as np
import datetime
import matplotlib.pyplot as plt

from helpers import *
from implementations import *
from Data_Cleaning import *
from training_procedure import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
initial_w = np.array([[0.5], [1.0]])
np.linalg.norm(initial_w), np.dot(initial_w.T, initial_w)

(1.118033988749895, array([[1.25]]))

In [127]:
initial_w = np.array([[0.5], [1.0]])
y = np.array([[0.1], [0.3], [0.5]])
tx = np.array([[2.3, 3.2], [1.0, 0.1], [1.4, 2.3]])
MAX_ITERS = 2
GAMMA = 0.1

expected_w = np.array([[0.463156], [0.939874]])
y = (y > 0.2) * 1.0
w, loss = logistic_regression(y, tx, expected_w, 0, GAMMA)
w, loss

(array([[0.463156],
        [0.939874]]),
 1.5336935954900122)

In [128]:
initial_w = np.array([[0.5], [1.0]])
y = np.array([[0.1], [0.3], [0.5]])
tx = np.array([[2.3, 3.2], [1.0, 0.1], [1.4, 2.3]])
MAX_ITERS = 2
GAMMA = 0.1

y = (y > 0.2) * 1.0
w, loss = logistic_regression(y, tx, initial_w, MAX_ITERS, GAMMA)
w, loss

(array([[0.37856053],
        [0.80113119]]),
 1.3483578175129305)

In [167]:
from helpers import *

initial_w = np.array([[0.5], [1.0]])
y = np.array([[0.1], [0.3], [0.5]])
tx = np.array([[2.3, 3.2], [1.0, 0.1], [1.4, 2.3]])
MAX_ITERS = 2
GAMMA = 0.1

lambda_ = 1.0
y = (y > 0.2) * 1.0
expected_w = np.array([[0.409111], [0.843996]])
w, loss = reg_logistic_regression(
    y, tx, lambda_, expected_w, 0, GAMMA
)
w, loss

(array([[0.409111],
        [0.843996]]),
 1.4073273893870055)

In [163]:
w_ = np.array([[0.409111], [0.843996]])
lambda_ = 1.0
lambda_*np.dot(w_.T, w_)

array([[0.87970106]])

In [166]:
from helpers import *

initial_w = np.array([[0.5], [1.0]])
y = np.array([[0.1], [0.3], [0.5]])
tx = np.array([[2.3, 3.2], [1.0, 0.1], [1.4, 2.3]])
MAX_ITERS = 2
GAMMA = 0.1

lambda_ = 1.0
y = (y > 0.2) * 1.0
w, loss = reg_logistic_regression(
    y, tx, lambda_, initial_w, MAX_ITERS, GAMMA
)
w, loss

(array([[0.21606187],
        [0.46774656]]),
 0.9721649929512526)

In [5]:
y, tx, ids = load_csv_data("train.csv", sub_sample=False)

In [107]:
tx.shape

(250000, 30)

In [108]:
def replaceFirstFeature(tx, initial_gamma):
    correct = tx[tx[:,0]!=-999][:,1:]
    ncorrect = tx[tx[:,0]==-999][:, 1:]

    # delete missing value columns 
    correct = np.delete(correct, [3, 4, 5, 11, 22, 23, 24, 25, 26, 27], axis = 1)
    ncorrect = np.delete(ncorrect, [3, 4, 5, 11, 22, 23, 24, 25, 26, 27], axis = 1)

    # transform the categorical features into one-hot features
    correct = np.hstack((correct, 
    (correct[:, 17] == 0).astype(np.float64)[:, np.newaxis],
    (correct[:, 17] == 1).astype(np.float64)[:, np.newaxis],
    (correct[:, 17] == 2).astype(np.float64)[:, np.newaxis],
    (correct[:, 17] == 3).astype(np.float64)[:, np.newaxis]))
    correct = np.delete(correct, 17, axis = 1)

    ncorrect = np.hstack((ncorrect, 
    (ncorrect[:, 17] == 0).astype(np.float64)[:, np.newaxis],
    (ncorrect[:, 17] == 1).astype(np.float64)[:, np.newaxis],
    (ncorrect[:, 17] == 2).astype(np.float64)[:, np.newaxis],
    (ncorrect[:, 17] == 3).astype(np.float64)[:, np.newaxis]))
    ncorrect = np.delete(ncorrect, 17, axis = 1)

    # # standardization
    # correct = standardize(correct)
    # ncorrect = standardize(ncorrect)

    # standardization
    correct[:, :-4] = standardize(correct[:, :-4])
    ncorrect[:, :-4] = standardize(ncorrect[:, :-4])

    num_feat = correct.shape[1]
    # add cross quadratic terms
    for i in range(num_feat):
        new_features_xx = np.multiply(correct[:, i:num_feat], correct[:, 0:num_feat-i])
        new_features_xxN = np.multiply(ncorrect[:, i:num_feat], ncorrect[:, 0:num_feat-i])
        xx = np.hstack([correct, new_features_xx])
        xxN = np.hstack([ncorrect, new_features_xxN])

    # # standardization
    # xx = standardize(xx)
    # xxN = standardize(xxN)

    # add the bias term 
    bias_term_xx = np.ones([xx.shape[0], 1])
    bias_term_xxN = np.ones([xxN.shape[0], 1])
    xx = np.hstack([bias_term_xx, xx])
    xxN = np.hstack([bias_term_xxN, xxN])

    w_init = np.zeros(xx.shape[1])
    best_w, best_loss = train_first_feature(tx[:,0][tx[:,0] != -999], xx, w_init, initial_gamma, 30, 0)
    newFirst = xxN.dot(best_w)
    print(f"Best loss: {best_loss}")
    tx[:,0][tx[:,0]==-999] = newFirst
    # tx[tx[:,0]==-999][:,0] = newFirst

    return tx

In [109]:
initial_gamma = 0.001
tx = replaceFirstFeature(tx, initial_gamma)

  3%|▎         | 1/30 [00:01<00:40,  1.40s/it]

Linear Regression SGD(0/29): loss=152.04840802599094


  7%|▋         | 2/30 [00:02<00:40,  1.44s/it]

Linear Regression SGD(1/29): loss=155.35716178706807


 10%|█         | 3/30 [00:04<00:38,  1.42s/it]

...

Linear Regression SGD(27/29): loss=150.7222346456378


 97%|█████████▋| 29/30 [00:50<00:02,  2.15s/it]

Linear Regression SGD(28/29): loss=150.72206435078525


100%|██████████| 30/30 [00:52<00:00,  1.76s/it]

Linear Regression SGD(29/29): loss=150.72210971098428
Best loss: 150.72206435078525


In [15]:
tx[tx[:,0]==-999][:,0], tx.shape

(array([], dtype=float64), (250000, 30))

In [19]:
model_training = split_according_num_split_jet(y, tx, ids)

In [20]:
tx_0 = model_training['0']['data']
tx_1 = model_training['1']['data']
tx_2 = model_training['2']['data']
tx_0.shape[0] + tx_1.shape[0] + tx_2.shape[0] == tx.shape[0]

True

In [21]:
tx_mean_meaningful_0 = np.zeros([tx_0.shape[1],])
for i in range(tx_0.shape[1]):
    tx_mean_meaningful_0[i] = np.mean(tx_0[:, i][tx_0[:, i] != -999])
    tx_0[:, i][tx_0[:, i] == -999] = tx_mean_meaningful_0[i]

In [ ]:
tx_mean_meaningful_1 = np.zeros([tx_1.shape[1],])
for i in range(tx_1.shape[1]):
    tx_mean_meaningful_1[i] = np.mean(tx_1[:, i][tx_1[:, i] != -999])
    tx_1[:, i][tx_1[:, i] == -999] = tx_mean_meaningful_1[i]

In [ ]:
tx_mean_meaningful_2 = np.zeros([tx_2.shape[1],])
for i in range(tx_2.shape[1]):
    tx_mean_meaningful_2[i] = np.mean(tx_2[:, i][tx_2[:, i] != -999])
    tx_2[:, i][tx_2[:, i] == -999] = tx_mean_meaningful_2[i]